# Notebook 06 — Incendio Florestal (WildFire)

**CLIMARISK-OG** | Petrobras | TRL5  
Ativo: REDUC — Refinaria Duque de Caxias, RJ  

Este notebook modela o risco de incendio florestal para a REDUC usando o modulo
nativo `WildFire` do `climada_petals`. Os dados de entrada sao sinteticos no
formato FIRMS/MODIS, calibrados com estatisticas de focos de calor do INPE
BDQueimadas para a regiao de Duque de Caxias (2015-2024).

**Impact function**: placeholder (linear simplificada). Sera substituida na
Etapa 2B com curva MDR fundamentada em literatura tecnica (SFPE Handbook,
API RP 752/753, Christou & Mattarelli 2000, ARIA/BARPI).

**Versao 1.0**

In [ ]:
!pip install climada climada-petals --quiet

In [ ]:
# BLOCO 1 -- Verificacao de Ambiente
import sys
print(f"Python: {sys.version}")

import climada
try:
    ver = climada.__version__
except AttributeError:
    from importlib.metadata import version
    ver = version("climada")
print(f"CLIMADA versao: {ver}")

# Verificar climada_petals (obrigatorio para WildFire)
try:
    from climada_petals.hazard import WildFire
    print("climada_petals.hazard.WildFire: OK")
    PETALS_WF_OK = True
except ImportError as e:
    print(f"ERRO: climada_petals WildFire nao disponivel: {e}")
    PETALS_WF_OK = False

CLIMADA_OK = True

In [ ]:
# =============================================================================
# BLOCO 2: IMPORTS
# =============================================================================
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib
from datetime import datetime
from scipy import sparse
import warnings
warnings.filterwarnings('ignore')

# CLIMADA core
from climada.hazard import Hazard, Centroids
from climada.entity import Exposures, ImpactFuncSet, ImpactFunc
from climada.engine import ImpactCalc
from climada.util.constants import ONE_LAT_KM

# CLIMADA petals -- WildFire
from climada_petals.hazard import WildFire

print("Imports: OK")

In [ ]:
# =============================================================================
# BLOCO 3: DEFINICAO DO ATIVO (REDUC)
# =============================================================================
# Mesmos parametros de NB01/NB02/NB03 para consistencia
REDUC_NAME = 'REDUC - Refinaria Duque de Caxias'
REDUC_LAT = -22.5167
REDUC_LON = -43.2833
REDUC_VALUE_USD = 5_000_000_000  # USD 5 bilhoes

# Bounding box para area de analise (mesma grade de NB01-NB03)
BBOX_BUFFER = 0.15  # graus (~17 km)
LAT_MIN = REDUC_LAT - BBOX_BUFFER
LAT_MAX = REDUC_LAT + BBOX_BUFFER
LON_MIN = REDUC_LON - BBOX_BUFFER
LON_MAX = REDUC_LON + BBOX_BUFFER

print(f"Ativo: {REDUC_NAME}")
print(f"Coordenadas: ({REDUC_LAT}, {REDUC_LON})")
print(f"Valor: USD {REDUC_VALUE_USD/1e9:.0f} bilhoes")
print(f"Bounding box: [{LAT_MIN:.4f}, {LAT_MAX:.4f}] x [{LON_MIN:.4f}, {LON_MAX:.4f}]")

In [ ]:
# =============================================================================
# BLOCO 4: EXPOSURES (E)
# =============================================================================
import geopandas as gpd
from shapely.geometry import Point

exp_gdf = gpd.GeoDataFrame(
    {
        'value': [REDUC_VALUE_USD],
        'latitude': [REDUC_LAT],
        'longitude': [REDUC_LON],
        'impf_WFsingle': [1],  # vinculo com impact function WildFire
    },
    geometry=[Point(REDUC_LON, REDUC_LAT)],
    crs='EPSG:4326'
)

exp = Exposures(exp_gdf)
exp.check()

print(f"Exposures: {len(exp.gdf)} ponto(s)")
print(f"Valor total: USD {exp.gdf['value'].sum():,.0f}")
print(f"Coluna de vinculo: impf_WFsingle = {exp.gdf['impf_WFsingle'].values[0]}")

In [ ]:
# =============================================================================
# BLOCO 5: GERACAO DE DADOS SINTETICOS FORMATO FIRMS/MODIS
# =============================================================================
# O modulo WildFire do climada_petals espera um DataFrame no formato FIRMS:
#   latitude, longitude, brightness, scan, track, acq_date, acq_time,
#   satellite, confidence, version, bright_t31, frp, daynight
#
# Calibracao baseada em dados do INPE BDQueimadas para Duque de Caxias:
#   - Media historica: ~15-40 focos/ano num raio de 30km da REDUC
#   - Concentracao: Jul-Out (periodo seco da Baixada Fluminense)
#   - Brightness MODIS tipico para vegetacao rasteira/degradada: 310-350 K
#   - FRP (Fire Radiative Power) tipico: 5-50 MW
#
# Geramos 10 anos sinteticos (2015-2024) com variabilidade interanual.

np.random.seed(42)

all_firms_rows = []
years = list(range(2015, 2025))

for year in years:
    # Numero de focos no ano (variabilidade interanual)
    n_fires = np.random.randint(10, 45)

    for _ in range(n_fires):
        # Posicao: espalhados na bounding box ao redor da REDUC
        lat = np.random.uniform(LAT_MIN, LAT_MAX)
        lon = np.random.uniform(LON_MIN, LON_MAX)

        # Data: concentrada no periodo seco (jul-out)
        month = np.random.choice([7, 8, 9, 10], p=[0.2, 0.35, 0.3, 0.15])
        day = np.random.randint(1, 29)
        acq_date = f"{year}-{month:02d}-{day:02d}"
        acq_time = np.random.choice([300, 600, 1200, 1500, 1800])

        # Brightness: temperatura de brilho MODIS (Kelvin)
        brightness = np.random.uniform(310, 370)

        # FRP: Fire Radiative Power (MW)
        frp = np.random.exponential(15) + 3  # media ~18 MW, minimo ~3

        # Scan/track (geometria do pixel MODIS)
        scan = np.random.uniform(1.0, 1.5)
        track = np.random.uniform(1.0, 1.2)

        # Bright_t31 (canal termal secundario)
        bright_t31 = brightness - np.random.uniform(10, 30)

        # Confianca e metadados
        confidence = np.random.choice([30, 50, 70, 80, 90], p=[0.05, 0.15, 0.3, 0.3, 0.2])

        all_firms_rows.append({
            'latitude': round(lat, 5),
            'longitude': round(lon, 5),
            'brightness': round(brightness, 2),
            'scan': round(scan, 2),
            'track': round(track, 2),
            'acq_date': acq_date,
            'acq_time': acq_time,
            'satellite': 'T',  # Terra
            'confidence': confidence,
            'version': '6.1',
            'bright_t31': round(bright_t31, 2),
            'frp': round(frp, 2),
            'daynight': np.random.choice(['D', 'N'], p=[0.6, 0.4]),
        })

firms_df = pd.DataFrame(all_firms_rows)
n_total_fires = len(firms_df)

print(f"Dados sinteticos FIRMS gerados: {n_total_fires} focos")
print(f"Periodo: {years[0]}-{years[-1]}")
print(f"Brightness range: {firms_df['brightness'].min():.0f} - {firms_df['brightness'].max():.0f} K")
print(f"FRP range: {firms_df['frp'].min():.1f} - {firms_df['frp'].max():.1f} MW")
print(f"\nDistribuicao por ano:")
firms_df['year'] = pd.to_datetime(firms_df['acq_date']).dt.year
print(firms_df.groupby('year').size().to_string())
print(f"\nPrimeiras 5 linhas:")
print(firms_df.head().to_string(index=False))

In [ ]:
# =============================================================================
# BLOCO 6: HAZARD (H) -- WildFire via climada_petals
# =============================================================================
# Tentativa 1: usar o modulo nativo WildFire.from_hist_fire_seasons_FIRMS
# Este metodo agrupa focos por ano (fire seasons) e cria eventos anuais.
#
# Se o modulo nativo falhar (dependencias GDAL, etc.), usamos fallback
# com classe Hazard generica preservando o haz_type 'WFsingle'.

USE_NATIVE_WF = False  # flag para rastrear qual metodo foi usado

try:
    print("Tentando WildFire nativo via climada_petals...")

    # Preparar DataFrame sem coluna auxiliar 'year'
    firms_input = firms_df.drop(columns=['year'], errors='ignore').copy()

    # Criar centroids na resolucao MODIS (~1 km)
    res_km = 1.0
    res_deg = res_km / ONE_LAT_KM
    centr_wf = Centroids.from_pnt_bounds(
        (LON_MIN, LAT_MIN, LON_MAX, LAT_MAX), res_deg
    )
    print(f"  Centroids: {centr_wf.size} pontos (resolucao ~{res_km} km)")

    # Usar from_hist_fire_seasons_FIRMS (metodo nao-depreciado)
    haz_wf = WildFire.from_hist_fire_seasons_FIRMS(
        firms_input,
        centroids=centr_wf,
        hemisphere='SHS'  # Hemisferio Sul: fire season Jul-Jun
    )

    USE_NATIVE_WF = True
    print(f"  WildFire nativo: OK")
    print(f"  Eventos (fire seasons): {haz_wf.size}")
    print(f"  Centroids: {haz_wf.centroids.size}")
    print(f"  Intensidade max: {haz_wf.intensity.max():.1f} (brightness, K)")
    print(f"  haz_type: {haz_wf.haz_type}")

except Exception as e:
    print(f"  WildFire nativo FALHOU: {e}")
    print("  Usando fallback com Hazard generico...")
    USE_NATIVE_WF = False

In [ ]:
# =============================================================================
# BLOCO 6B: FALLBACK -- Hazard generico se WildFire nativo falhar
# =============================================================================
# Este bloco so executa se o modulo nativo nao funcionou.
# Cria um Hazard generico com haz_type='WFsingle' e intensidade baseada
# no brightness maximo por fire season.
#
# Abordagem: cada ano e um evento. Intensidade nos centroids onde
# houve focos. Frequencia = 1/n_anos por evento.

if not USE_NATIVE_WF:
    print("Construindo Hazard WFsingle via fallback generico...")

    # Centroids: mesma grade dos outros notebooks
    n_centr_lat = int((LAT_MAX - LAT_MIN) / (1.0 / ONE_LAT_KM)) + 1
    n_centr_lon = int((LON_MAX - LON_MIN) / (1.0 / ONE_LAT_KM)) + 1
    lats = np.linspace(LAT_MIN, LAT_MAX, n_centr_lat)
    lons = np.linspace(LON_MIN, LON_MAX, n_centr_lon)
    lon_grid, lat_grid = np.meshgrid(lons, lats)
    centr_lat = lat_grid.flatten()
    centr_lon = lon_grid.flatten()
    n_centroids = len(centr_lat)

    centroids = Centroids(
        lat=centr_lat,
        lon=centr_lon,
    )

    # Agrupar focos por ano e calcular estatisticas
    firms_df['year'] = pd.to_datetime(firms_df['acq_date']).dt.year
    yearly_stats = firms_df.groupby('year').agg(
        n_fires=('brightness', 'count'),
        max_brightness=('brightness', 'max'),
        mean_frp=('frp', 'mean'),
        max_frp=('frp', 'max'),
    ).reset_index()

    n_events = len(yearly_stats)
    n_years = years[-1] - years[0] + 1

    # Construir matriz de intensidade
    intensity_matrix = np.zeros((n_events, n_centroids))
    fraction_matrix = np.zeros((n_events, n_centroids))

    for i, (_, row) in enumerate(yearly_stats.iterrows()):
        yr = int(row['year'])
        yr_fires = firms_df[firms_df['year'] == yr]

        # Para cada foco, encontrar centroid mais proximo e atribuir intensidade
        for _, fire in yr_fires.iterrows():
            dists = np.sqrt((centr_lat - fire['latitude'])**2 +
                           (centr_lon - fire['longitude'])**2)
            nearest_idx = np.argmin(dists)
            # Usar brightness como intensidade (acumular maximo)
            intensity_matrix[i, nearest_idx] = max(
                intensity_matrix[i, nearest_idx],
                fire['brightness']
            )
            fraction_matrix[i, nearest_idx] = 1.0

    # Frequencia: cada fire season ocorre 1x no periodo observado
    frequency = np.ones(n_events) / n_years

    # Event IDs e nomes
    event_id = np.arange(1, n_events + 1)
    event_name = [str(yr) for yr in yearly_stats['year'].values]

    haz_wf = Hazard(
        haz_type='WFsingle',
        centroids=centroids,
        event_id=event_id,
        event_name=event_name,
        frequency=frequency,
        intensity=sparse.csr_matrix(intensity_matrix),
        fraction=sparse.csr_matrix(fraction_matrix),
        units='K',
    )

    print(f"  Hazard WFsingle (fallback): OK")
    print(f"  Eventos: {haz_wf.size}")
    print(f"  Centroids: {haz_wf.centroids.size}")
    print(f"  Intensidade max: {haz_wf.intensity.max():.1f} K")
    print(f"  haz_type: {haz_wf.haz_type}")
    print(f"  Metodo: Hazard generico (fallback)")
else:
    n_events = haz_wf.size
    print("Bloco 6B ignorado -- WildFire nativo usado no Bloco 6.")

In [ ]:
# =============================================================================
# BLOCO 7: IMPACT FUNCTION (V) -- PLACEHOLDER
# =============================================================================
# STATUS: PLACEHOLDER -- sera substituida na Etapa 2B com curva MDR
# fundamentada em:
#   - SFPE Handbook of Fire Protection Engineering
#   - API RP 752/753 (Management of Hazards in process plants)
#   - Christou & Mattarelli (2000) -- Land-use planning near industrial sites
#   - ARIA/BARPI -- French industrial accident database
#
# Placeholder: funcao linear simples
#   - Intensidade: brightness (K) do MODIS
#   - Limiar: 300 K (abaixo = sem dano)
#   - Saturacao: 500 K (dano maximo = 30% do valor)
#   - MDR maximo: 0.30 (30%) -- conservador para infraestrutura industrial
#     com protecao contra incendio (brigada, hidrantes, sistemas fixos)
#
# NOTA: O MDR maximo de 30% reflete que refinarias possuem:
#   - Sistemas fixos de combate (sprinklers, monitores de espuma)
#   - Brigada industrial 24h
#   - Projeto de layout com distancias de seguranca (API 2510)
#   - Danos de incendio florestal sao predominantemente indiretos
#     (interrupcao, danos perifericos, contaminacao por fumaca)

IMPF_STATUS = 'placeholder'  # sera 'final' na Etapa 2B

# Pontos da curva MDR
intensity_points = np.array([0, 300, 350, 400, 450, 500])
mdr_points = np.array([0.0, 0.0, 0.05, 0.12, 0.22, 0.30])

impf_wf = ImpactFunc(
    id=1,
    haz_type='WFsingle',
    name='WildFire -- Industrial Facility (Placeholder)',
    intensity_unit='K',
    intensity=intensity_points,
    mdd=mdr_points,       # Mean Damage Degree
    paa=np.ones(len(intensity_points)),  # 100% do ativo exposto
)

impf_set = ImpactFuncSet([impf_wf])
impf_set.check()

print("Impact Function WildFire (PLACEHOLDER):")
print(f"  ID: {impf_wf.id}")
print(f"  haz_type: {impf_wf.haz_type}")
print(f"  Intensidade: {intensity_points[0]} - {intensity_points[-1]} K")
print(f"  MDR range: {mdr_points[0]:.0%} - {mdr_points[-1]:.0%}")
print(f"  Status: {IMPF_STATUS}")
print(f"  NOTA: Sera substituida na Etapa 2B com curva cientificamente fundamentada")

In [ ]:
# =============================================================================
# BLOCO 7B: VISUALIZACAO DA IMPACT FUNCTION
# =============================================================================
fig, ax = plt.subplots(1, 1, figsize=(8, 5))

ax.plot(intensity_points, mdr_points * 100, 'o-', color='#e67e22',
        linewidth=2, markersize=8, label='Placeholder (linear)')
ax.fill_between(intensity_points, 0, mdr_points * 100,
                alpha=0.15, color='#e67e22')

ax.axvline(x=300, color='gray', linestyle='--', alpha=0.5, label='Limiar (300 K)')
ax.axhline(y=30, color='red', linestyle=':', alpha=0.5, label='MDR max (30%)')

ax.set_xlabel('Brightness Temperature (K)', fontsize=12)
ax.set_ylabel('Mean Damage Ratio (%)', fontsize=12)
ax.set_title('Impact Function: WildFire -- Industrial Facility\n'
             '(PLACEHOLDER -- sera substituida na Etapa 2B)', fontsize=12)
ax.legend(fontsize=10)
ax.set_xlim(0, 550)
ax.set_ylim(0, 40)
ax.grid(True, alpha=0.3)

# Anotacao de status
ax.text(0.98, 0.95, 'STATUS: PLACEHOLDER',
        transform=ax.transAxes, fontsize=11, fontweight='bold',
        color='red', ha='right', va='top',
        bbox=dict(boxstyle='round', facecolor='#fff3cd', alpha=0.9))

plt.tight_layout()
plt.savefig('nb06_impact_function_wf.png', dpi=150, bbox_inches='tight')
plt.show()
print("  OK - Salvo: nb06_impact_function_wf.png")

In [ ]:
# =============================================================================
# BLOCO 8: CALCULO DE IMPACTO (H x E x V)
# =============================================================================
print("Calculando impacto WildFire...")
print(f"  Hazard: {haz_wf.size} eventos, {haz_wf.centroids.size} centroids")
print(f"  Exposure: {len(exp.gdf)} ponto(s), USD {exp.gdf['value'].sum():,.0f}")
print(f"  Impact function: {impf_wf.name}")

imp_wf = ImpactCalc(exp, impf_set, haz_wf).impact(save_mat=True)

eai_wf = float(imp_wf.eai_exp.sum())
eai_ratio_wf = (eai_wf / REDUC_VALUE_USD) * 100

print(f"\n  EAI (Expected Annual Impact): USD {eai_wf:,.0f}")
print(f"  EAI / Valor do ativo: {eai_ratio_wf:.2f}%")
print(f"  Impacto por evento (fire season):")

for i in range(min(n_events, 10)):
    evt_name = haz_wf.event_name[i] if hasattr(haz_wf, 'event_name') and len(haz_wf.event_name) > i else f'Evt {i+1}'
    print(f"    {evt_name}: USD {imp_wf.at_event[i]:>15,.0f}")

In [ ]:
# =============================================================================
# BLOCO 9: VISUALIZACOES
# =============================================================================
fig, axes = plt.subplots(2, 2, figsize=(14, 10))

# --- Painel 1: Mapa de intensidade maxima ---
try:
    ax = axes[0, 0]
    max_int = np.array(haz_wf.intensity.max(axis=0).todense()).flatten()
    centr_lats = haz_wf.centroids.lat
    centr_lons = haz_wf.centroids.lon
    mask = max_int > 0
    if mask.sum() > 0:
        sc = ax.scatter(centr_lons[mask], centr_lats[mask], c=max_int[mask],
                       cmap='YlOrRd', s=15, vmin=300, vmax=400)
        plt.colorbar(sc, ax=ax, label='Brightness (K)')
    ax.plot(REDUC_LON, REDUC_LAT, 'k^', markersize=12, markeredgewidth=2,
            label='REDUC')
    ax.set_xlabel('Longitude')
    ax.set_ylabel('Latitude')
    ax.set_title('Intensidade Maxima WildFire', fontsize=11, fontweight='bold')
    ax.legend(fontsize=9)
    ax.grid(True, alpha=0.3)
except Exception as e:
    axes[0, 0].text(0.5, 0.5, f'Erro: {e}', transform=axes[0, 0].transAxes,
                    ha='center', va='center')

# --- Painel 2: Focos por ano ---
try:
    ax = axes[0, 1]
    yearly_counts = firms_df.groupby('year').size()
    ax.bar(yearly_counts.index, yearly_counts.values, color='#e67e22', alpha=0.8)
    ax.set_xlabel('Ano')
    ax.set_ylabel('Numero de focos')
    ax.set_title('Focos de Calor por Ano (sintetico)', fontsize=11, fontweight='bold')
    ax.grid(True, alpha=0.3, axis='y')
except Exception as e:
    axes[0, 1].text(0.5, 0.5, f'Erro: {e}', transform=axes[0, 1].transAxes,
                    ha='center', va='center')

# --- Painel 3: Distribuicao de brightness ---
try:
    ax = axes[1, 0]
    ax.hist(firms_df['brightness'], bins=20, color='#e74c3c', alpha=0.7,
            edgecolor='black', linewidth=0.5)
    ax.axvline(x=300, color='gray', linestyle='--', label='Limiar impf (300 K)')
    ax.set_xlabel('Brightness Temperature (K)')
    ax.set_ylabel('Frequencia')
    ax.set_title('Distribuicao de Brightness (FIRMS)', fontsize=11, fontweight='bold')
    ax.legend(fontsize=9)
    ax.grid(True, alpha=0.3, axis='y')
except Exception as e:
    axes[1, 0].text(0.5, 0.5, f'Erro: {e}', transform=axes[1, 0].transAxes,
                    ha='center', va='center')

# --- Painel 4: Impacto por fire season ---
try:
    ax = axes[1, 1]
    evt_labels = []
    for i in range(n_events):
        if hasattr(haz_wf, 'event_name') and len(haz_wf.event_name) > i:
            evt_labels.append(str(haz_wf.event_name[i]))
        else:
            evt_labels.append(f'{years[i]}')
    impacts_m = imp_wf.at_event / 1e6
    bars = ax.bar(range(n_events), impacts_m, color='#d35400', alpha=0.8)
    ax.set_xticks(range(n_events))
    ax.set_xticklabels(evt_labels, rotation=45, fontsize=8)
    ax.set_xlabel('Fire Season')
    ax.set_ylabel('Impacto (USD milhoes)')
    ax.set_title('Impacto por Fire Season', fontsize=11, fontweight='bold')
    ax.grid(True, alpha=0.3, axis='y')

    # Linha de EAI
    ax.axhline(y=eai_wf/1e6, color='red', linestyle='--', linewidth=1.5,
               label=f'EAI = USD {eai_wf/1e6:,.0f}M')
    ax.legend(fontsize=9)
except Exception as e:
    axes[1, 1].text(0.5, 0.5, f'Erro: {e}', transform=axes[1, 1].transAxes,
                    ha='center', va='center')

plt.suptitle('CLIMARISK-OG | NB06 | WildFire -- REDUC', fontsize=14,
             fontweight='bold', y=1.02)
plt.tight_layout()
plt.savefig('nb06_wildfire_panel.png', dpi=150, bbox_inches='tight')
plt.show()
print("  OK - Painel salvo: nb06_wildfire_panel.png")

In [ ]:
# =============================================================================
# BLOCO 10: RESUMO EXECUTIVO
# =============================================================================
hazard_method = 'WildFire nativo (climada_petals)' if USE_NATIVE_WF else 'Hazard generico (fallback)'

print("\n" + "=" * 70)
print("  RESUMO EXECUTIVO -- CLIMARISK-OG | Notebook 06 | WildFire")
print("=" * 70)
print(f"""
  ATIVO:       {REDUC_NAME}
  LOCALIZACAO: Duque de Caxias, RJ ({REDUC_LAT}, {REDUC_LON})
  VALOR:       USD {REDUC_VALUE_USD/1e9:.0f} bilhoes
  CENARIO:     Baseline (historico sintetico 2015-2024)
  HAZARD:      Incendio Florestal (WildFire)

  RESULTADOS PRINCIPAIS:
  ---------------------------------------------------
  EAI WildFire:          USD {eai_wf:>15,.0f}
  Ratio EAI/Valor:       {eai_ratio_wf:>14.2f}%
  ---------------------------------------------------

  DADOS DE ENTRADA:
    Formato: FIRMS/MODIS (sintetico)
    Calibracao: INPE BDQueimadas, Duque de Caxias (2015-2024)
    Focos totais: {n_total_fires}
    Fire seasons: {n_events}
    Metodo hazard: {hazard_method}

  FUNCAO DE DANO:
    Nome: {impf_wf.name}
    Status: {IMPF_STATUS.upper()}
    Intensidade: brightness (K)
    Limiar: 300 K | Saturacao: 500 K | MDR max: 30%
    Proxima etapa: curva MDR cientifica (Etapa 2B)
    Referencias pendentes: SFPE, API RP 752/753, Christou & Mattarelli,
                           ARIA/BARPI

  LIMITACOES (TRL5):
    1. Dados de hazard sinteticos (nao FIRMS real)
    2. Valor de exposicao estimado
    3. Impact function PLACEHOLDER (nao calibrada)
    4. Cenario baseline apenas
    5. Ativo unico
    6. Incendio florestal modelado como proxy (brightness)
       e nao como propagacao fisica de fogo

======================================================================
  FIM DO NOTEBOOK 06 -- Versao 1.0
======================================================================
""")

In [ ]:
# =============================================================================
# BLOCO 11: EXPORTACAO JSON
# =============================================================================
import json

results = {
    "metadata": {
        "notebook": "nb06_wildfire_duque_caxias",
        "version": "1.0",
        "date": datetime.now().isoformat(),
        "climada_version": ver,
        "methodology": "CLIMADA H x E x V probabilistic impact",
        "n_hazards": 1
    },
    "asset": {
        "name": REDUC_NAME,
        "lat": REDUC_LAT,
        "lon": REDUC_LON,
        "value_usd": REDUC_VALUE_USD
    },
    "hazards": {
        "WF": {
            "type": "WFsingle",
            "type_name": "WildFire",
            "n_events": n_events,
            "event_names": [str(n) for n in haz_wf.event_name] if hasattr(haz_wf, 'event_name') else [str(y) for y in years],
            "intensity_unit": "K (brightness)",
            "max_intensity": float(haz_wf.intensity.max()),
            "data_source": {
                "format": "FIRMS/MODIS (synthetic)",
                "calibration": "INPE BDQueimadas, Duque de Caxias (2015-2024)",
                "n_hotspots": n_total_fires,
                "period": f"{years[0]}-{years[-1]}"
            },
            "hazard_method": "WildFire nativo (climada_petals)" if USE_NATIVE_WF else "Hazard generico (fallback)",
            "impact_function": {
                "name": impf_wf.name,
                "id": int(impf_wf.id),
                "status": IMPF_STATUS,
                "source": "Placeholder -- linear simplificada",
                "pending_references": [
                    "SFPE Handbook of Fire Protection Engineering",
                    "API RP 752/753",
                    "Christou & Mattarelli (2000)",
                    "ARIA/BARPI database"
                ],
                "type": "structural_and_operational_damage",
                "threshold_K": 300,
                "saturation_K": 500,
                "mdr_max": 0.30
            },
            "results": {
                "eai_usd": eai_wf,
                "eai_ratio_pct": eai_ratio_wf,
                "impact_by_event": {
                    str(haz_wf.event_name[i] if hasattr(haz_wf, 'event_name') and len(haz_wf.event_name) > i else years[i]):
                    float(imp_wf.at_event[i])
                    for i in range(n_events)
                }
            }
        }
    },
    "limitations": [
        "Dados de hazard sinteticos (nao FIRMS real)",
        "Valor de exposicao estimado",
        "Impact function PLACEHOLDER (nao calibrada cientificamente)",
        "Cenario baseline apenas",
        "Ativo unico",
        "Brightness como proxy de severidade (nao propagacao fisica)"
    ]
}

with open('results_nb06_wildfire_reduc.json', 'w', encoding='utf-8') as f:
    json.dump(results, f, indent=2, ensure_ascii=False)

print("\nJSON exportado: results_nb06_wildfire_reduc.json")
print("\nConteudo resumido:")
print(f"  EAI: USD {results['hazards']['WF']['results']['eai_usd']:,.0f}")
print(f"  Impact function status: {results['hazards']['WF']['impact_function']['status']}")
print(f"  Metodo hazard: {results['hazards']['WF']['hazard_method']}")

In [ ]:
# =============================================================================
# BLOCO 12: DIAGNOSTICO E ARTEFATOS
# =============================================================================
import os
from IPython.display import display, Image

print("=" * 60)
print("  PARTE 1: ARTEFATOS GERADOS")
print("=" * 60)

output_files = [
    'nb06_impact_function_wf.png',
    'nb06_wildfire_panel.png',
    'results_nb06_wildfire_reduc.json',
]

for f in output_files:
    if os.path.exists(f):
        size = os.path.getsize(f) / 1024
        print(f"  OK {f} ({size:.1f} KB)")
        if f.endswith('.png'):
            display(Image(filename=f, width=700))
    else:
        print(f"  FALTA {f}")

print("\n" + "=" * 60)
print("  PARTE 2: DIAGNOSTICO DE CONSISTENCIA")
print("=" * 60)

checks = []

# Check 1: EAI > 0
ok = eai_wf > 0
checks.append(('EAI WildFire > 0', ok, f'USD {eai_wf:,.0f}'))

# Check 2: Impactos nao excedem valor do ativo
max_impact = max(imp_wf.at_event)
ok = max_impact <= REDUC_VALUE_USD * 1.01
checks.append(('Impacto <= valor do ativo', ok, f'max={max_impact:,.0f}'))

# Check 3: haz_type correto
ok = haz_wf.haz_type == 'WFsingle'
checks.append(('haz_type = WFsingle', ok, haz_wf.haz_type))

# Check 4: Impact function e placeholder
ok = IMPF_STATUS == 'placeholder'
checks.append(('Impact function = placeholder', ok, IMPF_STATUS))

# Check 5: JSON exportado
ok = os.path.exists('results_nb06_wildfire_reduc.json')
checks.append(('JSON exportado', ok, ''))

# Check 6: JSON contem campo status
if ok:
    with open('results_nb06_wildfire_reduc.json', 'r') as jf:
        j = json.load(jf)
    ok = j['hazards']['WF']['impact_function']['status'] == 'placeholder'
    checks.append(('JSON marca impf como placeholder', ok, ''))

# Check 7: Dados FIRMS gerados
ok = n_total_fires > 0
checks.append(('Dados FIRMS sinteticos gerados', ok, f'{n_total_fires} focos'))

# Check 8: Coordenadas consistentes com NB01-NB03
ok = (abs(exp.gdf['latitude'].values[0] - REDUC_LAT) < 0.001 and
      abs(exp.gdf['longitude'].values[0] - REDUC_LON) < 0.001)
checks.append(('Coordenadas REDUC consistentes', ok,
               f'{exp.gdf["latitude"].values[0]}, {exp.gdf["longitude"].values[0]}'))

for name, passed, detail in checks:
    status = 'OK' if passed else 'FALHA'
    print(f"  [{status}] {name}: {detail}")

n_passed = sum(1 for _, p, _ in checks if p)
print(f"\n  Resultado: {n_passed}/{len(checks)} checks passaram")

if n_passed == len(checks):
    print("  Notebook pronto para commit no GitHub.")
else:
    print("  ATENCAO: Verificar checks que falharam antes de commitar.")

# Mostrar conteudo do JSON
print("\nConteudo do JSON:")
with open('results_nb06_wildfire_reduc.json', 'r') as jf:
    print(json.dumps(json.load(jf), indent=2, ensure_ascii=False))